# 📈 Integrated Dividend Feature Analysis

This notebook combines visual correlation analysis, regression diagnostics, and model-based feature importance to analyze the impact of various features on dividend-related metrics.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score

sns.set_theme(style="whitegrid")
plt.rcParams['figure.figsize'] = (10, 6)

In [ ]:
# --- Configuration ---
import glob

# Load all ticker history files
history_files = glob.glob("features_data/tickers_history/*.parquet")
df_features = pd.concat([pd.read_parquet(f) for f in history_files], ignore_index=True)

# Load macroeconomic data
df_macro = pd.read_parquet("features_data/macro_history/united_states.parquet")

# Load static sector/country info and merge
df_static = pd.read_parquet("features_data/tickers_static/static_ticker_info.parquet")
df_features = df_features.merge(df_static, on="ticker", how="left")


DIVIDEND_FEATURES = [
    'dividend_yield',
    'dividend_cagr_3y',
    'dividend_cagr_5y',
    'yield_vs_5y_median'
]

BINARY_FEATURES = [
    'sector_technology', 'sector_healthcare', 'sector_utilities',
    'sector_basic_materials', 'sector_financial_services', 'sector_consumer_cyclical',
    'sector_real_estate', 'sector_communication_services', 'sector_industrials',
    'sector_energy', 'sector_materials', 'sector_consumer_defensive',
    'ebit_interest_cover_capped',
    'has_eps_cagr_3y', 'has_fcf_cagr_3y', 'has_dividend_yield',
    'has_dividend_cagr_3y', 'has_dividend_cagr_5y', 'has_ebit_interest_cover'
]

EXCLUDE_FEATURES = ['ticker', 'as_of', 'as_of_year', 'country'] + DIVIDEND_FEATURES

print("Loading data...")

df_features['as_of'] = pd.to_datetime(df_features['as_of'])
df_features['as_of_year'] = df_features['as_of'].dt.year
df_features['as_of_year'] = df_features['as_of_year'].astype(df_macro['as_of_year'].dtype)

df_merged = pd.merge(
    df_features,
    df_macro,
    how='left',
    on=['as_of_year', 'country']
)

print(df_merged['gdp_yoy_backfilled'].value_counts(dropna=False))

df_merged.drop(columns=['backfilled_year'], errors='ignore', inplace=True)
print("Data loaded and merged successfully.")

In [ ]:
print("df_features countries:", df_features['country'].unique())
print("df_macro countries:", df_macro['country'].unique())

In [ ]:
all_columns = df_merged.columns.tolist()
independent_features = [col for col in all_columns if col not in EXCLUDE_FEATURES]
numerical_independent_features = [f for f in independent_features if f not in BINARY_FEATURES]
categorical_independent_features = [f for f in independent_features if f in BINARY_FEATURES]

In [ ]:
predictor_categories = {
    'Financial_Metrics': [f for f in numerical_independent_features if 'return' in f or 'ratio' in f or 'cagr' in f or 'cover' in f],
    'Macro_Economic': ['gdp_yoy_backfilled', 'inflation_latest', 'unemployment_latest', 'consumption_backfilled'],
    'Sector_Features': [f for f in df_merged.columns if f.startswith('sector_')]
}

In [ ]:
# def analyze_feature_impact(target_feature, max_n_plots=10, max_n_importance=10):
#     df_analysis = df_merged[df_merged[target_feature].notna()].copy()
#     if df_analysis.empty or df_analysis[target_feature].nunique() <= 1:
#         print(f"⚠️ '{target_feature}' has insufficient variation. Skipping.")
#         return

#     print(f"\n🔍 Analyzing: {target_feature}")

#     # === Scatter + Regression plots ===
#     for feature in numerical_independent_features:
#         if (
#             feature in df_analysis.columns
#             and df_analysis[feature].notna().sum() >= 3
#             and df_analysis[feature].nunique() > 2
#             and not feature.endswith('_capped')
#             and not feature.startswith('has_')
#         ):
#             plt.figure(figsize=(7, 4))
#             sns.scatterplot(x=feature, y=target_feature, data=df_analysis, alpha=0.6)
#             sns.regplot(x=feature, y=target_feature, data=df_analysis, scatter=False, color='red', line_kws={'alpha': 0.6})
#             plt.title(f'{target_feature} vs. {feature}')
#             plt.tight_layout()
#             plt.show()

#     # === One-hot sector to single column ===
#     sector_columns = [f for f in df_analysis.columns if f.startswith("sector_")]
#     if sector_columns:
#         df_analysis['sector'] = df_analysis[sector_columns].idxmax(axis=1).str.replace('sector_', '').str.title()
#         df_analysis.drop(columns=sector_columns, inplace=True)
#         if df_analysis['sector'].nunique() > 1:
#             plt.figure(figsize=(10, 5))
#             sns.boxplot(x='sector', y=target_feature, data=df_analysis)
#             plt.title(f'{target_feature} distribution by sector')
#             plt.xticks(rotation=45)
#             plt.tight_layout()
#             plt.show()

#     # === Boxplots for binary categorical features ===
#     for feature in categorical_independent_features:
#         if (
#             feature in df_analysis.columns
#             and not feature.startswith('has_')
#             and not feature.endswith('_capped')
#         ):
#             df_analysis[feature] = df_analysis[feature].astype('category')
#             if df_analysis[feature].nunique() <= 1:
#                 continue
#             value_counts = df_analysis[feature].value_counts(normalize=True)
#             if value_counts.iloc[0] > 0.95:
#                 continue  # Skip if imbalanced
#             plt.figure(figsize=(7, 4))
#             sns.boxplot(x=feature, y=target_feature, data=df_analysis)
#             plt.title(f'{target_feature} distribution by {feature}')
#             plt.tight_layout()
#             plt.show()

#     # === Correlation Bar Plots ===
#     fig, axes = plt.subplots(1, len(predictor_categories), figsize=(5 * len(predictor_categories), 4))
#     if len(predictor_categories) == 1:
#         axes = [axes]

#     for ax, (group_name, group_features) in zip(axes, predictor_categories.items()):
#         valid_feats = [
#             f for f in group_features if (
#                 f in df_analysis.columns and 
#                 df_analysis[f].notna().sum() > 2 and 
#                 not f.startswith('has_') and 
#                 not f.endswith('_capped') and 
#                 not f.startswith('sector_')
#             )
#         ]
#         if not valid_feats:
#             ax.set_visible(False)
#             continue
#         data_for_corr = df_analysis[valid_feats + [target_feature]].dropna()
#         if data_for_corr.shape[0] < 3:
#             ax.set_visible(False)
#             continue
#         corr_vals = data_for_corr.corr()[target_feature].drop(target_feature).dropna()
#         corr_vals = corr_vals.sort_values(key=np.abs, ascending=False).head(max_n_plots)
#         colors = ['red' if v < 0 else 'blue' for v in corr_vals]
#         ax.barh(corr_vals.index.str.replace('_', ' ').str.title(), corr_vals.values, color=colors)
#         ax.set_title(f"{group_name} Correlation")
#         ax.axvline(0, color='black', lw=0.8)
#         ax.grid(True, linestyle='--', alpha=0.5)

#     plt.tight_layout()
#     plt.show()

#     # === Random Forest Feature Importance ===
#     rf_features = [
#         f for group in predictor_categories.values() for f in group if (
#             f in df_analysis.columns and
#             not f.endswith('_capped') and
#             not f.startswith('has_') and
#             not f.startswith('sector_')
#         )
#     ]
#     model_data = df_analysis[rf_features + [target_feature]].dropna()
#     if len(model_data) < 10:
#         print("Not enough data for modeling.")
#         return

#     X = model_data[rf_features].replace([np.inf, -np.inf], np.nan).dropna()
#     y = model_data.loc[X.index, target_feature]

#     if X.shape[1] < 2:
#         print("⚠️ Too few features with valid data for modeling.")
#         return

#     rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
#     rf_model.fit(X, y)
#     importance_df = pd.DataFrame({
#         'Feature': X.columns,
#         'Importance': rf_model.feature_importances_
#     }).sort_values('Importance', ascending=False).head(max_n_importance)

#     # === Print top correlations ===
#     numeric_cols = df_analysis.select_dtypes(include=[np.number]).columns
#     if target_feature not in numeric_cols:
#         print(f"⚠️ '{target_feature}' is not numeric. Skipping correlation summary.")
#     else:
#         correlation_series = df_analysis[numeric_cols].corr()[target_feature].drop(target_feature).dropna()
#         top_corrs = correlation_series.abs().sort_values(ascending=False).head(5)
#         print(f"📌 Top correlations with {target_feature}:\n{top_corrs}")

#     # === Plot Feature Importance ===
#     fig, ax = plt.subplots(figsize=(8, 0.3 * len(importance_df)))
#     colors = plt.cm.viridis(np.linspace(0, 1, len(importance_df)))
#     bars = ax.barh(range(len(importance_df)), importance_df['Importance'], color=colors)

#     ax.set_yticks(range(len(importance_df)))
#     ax.set_yticklabels([f.replace('_', ' ').title() for f in importance_df['Feature']])
#     ax.set_xlabel('Feature Importance (Random Forest)')
#     ax.set_title(f'Feature Importance for Predicting {target_feature.replace("_", " ").title()}')
#     ax.grid(True, alpha=0.3)

#     for bar, importance in zip(bars, importance_df['Importance']):
#         ax.text(importance + 0.001, bar.get_y() + bar.get_height() / 2,
#                 f'{importance:.3f}', ha='left', va='center', fontsize=8)

#     plt.tight_layout()
#     plt.show()

#     print(f"Model R² Score: {r2_score(y, rf_model.predict(X)):.3f}")


In [ ]:
def plot_scatter_regression(df, target_feature, numerical_features):
    for feature in numerical_features:
        if (
            feature in df.columns
            and df[feature].notna().sum() >= 3
            and df[feature].nunique() > 2
            and not feature.endswith('_capped')
            and not feature.startswith('has_')
        ):
            plt.figure(figsize=(7, 4))
            sns.scatterplot(x=feature, y=target_feature, data=df, alpha=0.6)
            sns.regplot(x=feature, y=target_feature, data=df, scatter=False, color='red', line_kws={'alpha': 0.6})
            plt.title(f'{target_feature} vs. {feature}')
            plt.tight_layout()
            plt.show()

def plot_sector_boxplot(df, target_feature):
    sector_columns = [f for f in df.columns if f.startswith("sector_")]
    if sector_columns:
        df['sector'] = df[sector_columns].idxmax(axis=1).str.replace('sector_', '').str.title()
        df.drop(columns=sector_columns, inplace=True)
        if df['sector'].nunique() > 1:
            plt.figure(figsize=(10, 5))
            sns.boxplot(x='sector', y=target_feature, data=df)
            plt.title(f'{target_feature} distribution by sector')
            plt.xticks(rotation=45)
            plt.tight_layout()
            plt.show()
    return df

def plot_binary_categorical_boxplots(df, target_feature, categorical_features):
    for feature in categorical_features:
        if (
            feature in df.columns
            and not feature.startswith('has_')
            and not feature.endswith('_capped')
        ):
            df[feature] = df[feature].astype('category')
            if df[feature].nunique() <= 1:
                continue
            value_counts = df[feature].value_counts(normalize=True)
            if value_counts.iloc[0] > 0.95:
                continue
            plt.figure(figsize=(7, 4))
            sns.boxplot(x=feature, y=target_feature, data=df)
            plt.title(f'{target_feature} distribution by {feature}')
            plt.tight_layout()
            plt.show()

def plot_correlations(df, target_feature, predictor_categories, max_n_plots):
    fig, axes = plt.subplots(1, len(predictor_categories), figsize=(5 * len(predictor_categories), 4))
    if len(predictor_categories) == 1:
        axes = [axes]

    for ax, (group_name, group_features) in zip(axes, predictor_categories.items()):
        valid_feats = [
            f for f in group_features if (
                f in df.columns and 
                df[f].notna().sum() > 2 and 
                not f.startswith('has_') and 
                not f.endswith('_capped') and 
                not f.startswith('sector_')
            )
        ]
        if not valid_feats:
            ax.set_visible(False)
            continue
        data_for_corr = df[valid_feats + [target_feature]].dropna()
        if data_for_corr.shape[0] < 3:
            ax.set_visible(False)
            continue
        corr_vals = data_for_corr.corr()[target_feature].drop(target_feature).dropna()
        corr_vals = corr_vals.sort_values(key=np.abs, ascending=False).head(max_n_plots)
        colors = ['red' if v < 0 else 'blue' for v in corr_vals]
        ax.barh(corr_vals.index.str.replace('_', ' ').str.title(), corr_vals.values, color=colors)
        ax.set_title(f"{group_name} Correlation")
        ax.axvline(0, color='black', lw=0.8)
        ax.grid(True, linestyle='--', alpha=0.5)

    plt.tight_layout()
    plt.show()

def plot_random_forest_importance(df, target_feature, predictor_categories, max_n_importance):
    rf_features = [
        f for group in predictor_categories.values() for f in group if (
            f in df.columns and
            not f.endswith('_capped') and
            not f.startswith('has_') and
            not f.startswith('sector_')
        )
    ]
    model_data = df[rf_features + [target_feature]].dropna()
    if len(model_data) < 10:
        print("Not enough data for modeling.")
        return

    X = model_data[rf_features].replace([np.inf, -np.inf], np.nan).dropna()
    y = model_data.loc[X.index, target_feature]

    if X.shape[1] < 2:
        print("⚠️ Too few features with valid data for modeling.")
        return

    rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
    rf_model.fit(X, y)
    importance_df = pd.DataFrame({
        'Feature': X.columns,
        'Importance': rf_model.feature_importances_
    }).sort_values('Importance', ascending=False).head(max_n_importance)

    fig, ax = plt.subplots(figsize=(8, 0.3 * len(importance_df)))
    colors = plt.cm.viridis(np.linspace(0, 1, len(importance_df)))
    bars = ax.barh(range(len(importance_df)), importance_df['Importance'], color=colors)

    ax.set_yticks(range(len(importance_df)))
    ax.set_yticklabels([f.replace('_', ' ').title() for f in importance_df['Feature']])
    ax.set_xlabel('Feature Importance (Random Forest)')
    ax.set_title(f'Feature Importance for Predicting {target_feature.replace("_", " ").title()}')
    ax.grid(True, alpha=0.3)

    for bar, importance in zip(bars, importance_df['Importance']):
        ax.text(importance + 0.001, bar.get_y() + bar.get_height() / 2,
                f'{importance:.3f}', ha='left', va='center', fontsize=8)

    plt.tight_layout()
    plt.show()

    print(f"Model R² Score: {r2_score(y, rf_model.predict(X)):.3f}")
    return importance_df


In [ ]:
def analyze_feature_impact(target_feature):
    df_analysis = df_merged[df_merged[target_feature].notna()].copy()
    if df_analysis.empty or df_analysis[target_feature].nunique() <= 1:
        print(f"⚠️ '{target_feature}' has insufficient variation. Skipping.")
        return

    print(f"\n🔍 Analyzing: {target_feature}")
    plot_scatter_regression(df_analysis, target_feature, numerical_independent_features)
    df_analysis = plot_sector_boxplot(df_analysis, target_feature)
    plot_binary_categorical_boxplots(df_analysis, target_feature, categorical_independent_features)
    plot_correlations(df_analysis, target_feature, predictor_categories, max_n_plots=10)
    plot_random_forest_importance(df_analysis, target_feature, predictor_categories, max_n_importance=10)


In [ ]:
for target in DIVIDEND_FEATURES:
    print (f"\n\n{'=' * 40}\nAnalyzing feature: {target}\n{'=' * 40}")
    analyze_feature_impact(target)


In [ ]:
# for target in DIVIDEND_FEATURES:
#     analyze_feature_impact(target)

### 🔥 Partial Correlation Heatmap using valid features only
🔍 What this does:
- Computes pairwise correlations.
- Selects only features with at least one absolute correlation ≥ 0.3 to a dividend feature.
- Shows a cleaner, interpretable heatmap using clustermap (which groups similar variables).

In [ ]:
correlation_features = DIVIDEND_FEATURES + numerical_independent_features
valid_corr_features = [
    col for col in correlation_features
    if col in df_merged.columns and df_merged[col].dtype in ['float64', 'float32', 'int64']
]

# Keep only columns with ≥5 non-NaN values
valid_counts = df_merged[valid_corr_features].notna().sum()
selected_features = valid_counts[valid_counts >= 5].index.tolist()

# Configuration
target_features = DIVIDEND_FEATURES
threshold = 0.3  # Adjust this threshold as needed

# Compute correlation matrix
corr_matrix = df_merged[selected_features].corr()

# Identify features with at least one strong correlation with any dividend feature
strong_corrs = corr_matrix[target_features].abs().max(axis=1)
filtered_features = strong_corrs[strong_corrs >= threshold].index.tolist()

# If nothing is found, fallback to a warning
if len(filtered_features) < 2:
    print(f"⚠️ No features found with |correlation| ≥ {threshold} to any dividend feature.")
else:
    # Compute filtered correlation matrix
    filtered_corr = df_merged[filtered_features].corr()

    # Optional: cluster to improve interpretability
    try:
        import seaborn as sns
        import matplotlib.pyplot as plt

        sns.clustermap(
            filtered_corr,
            annot=True,
            fmt=".2f",
            cmap="coolwarm",
            center=0,
            linewidths=0.5,
            figsize=(12, 10)
        )
        plt.title(f"Filtered Correlation Heatmap (|r| ≥ {threshold})", pad=80)
        plt.show()
    except Exception as e:
        print(f"⚠️ Failed to render heatmap: {e}")

### 📌 **Top Insights on Dividend Features**

#### 1. **Dividend Yield**

* Strong **positive correlation** with:

  * `payout_ratio` (r ≈ 0.73)
  * `net_debt_to_ebitda` (r ≈ 0.57)
* Strong **negative correlation** with:

  * `sma_50_200_delta`, `fcf_cagr_3y`, and `volatility` (r ≈ -0.5 to -0.6)

🧠 *Interpretation*: High dividend yield tends to occur in **financially mature, lower-growth stocks** with higher debt and slower momentum.

---

#### 2. **Dividend CAGR (3Y, 5Y)**

* Strong **positive correlation** with:

  * `fcf_cagr_3y`, `eps_cagr_3y` (r ≈ 0.8+)
  * `sma_50_200_delta`, `6m_return`, `12m_return` (r ≈ 0.5–0.8)

🧠 *Interpretation*: Sustained dividend growth is **highly aligned with earnings and cash flow growth**, and coincides with **positive price momentum**.

---

#### 3. **Yield vs. 5Y Median**

* Strong **negative correlation** with:

  * `fcf_cagr_3y`, `dividend_cagr_5y`, `sma_50_200_delta` (r ≈ -0.5 to -0.8)

🧠 *Interpretation*: When dividend growth and performance are high, yields tend to be **compressed vs. historical median**, likely due to price appreciation.

---

### 💥 Additional Highlights

* `fcf_cagr_3y` and `eps_cagr_3y` are **core growth drivers**, highly correlated with most dividend-growth indicators.
* `sector_relative_6m` and `sma_50_200_delta` are also informative of dividend trends, linking **technical momentum** to dividend behaviors.
* `net_debt_to_ebitda` shows up as a red flag for **high yield but low growth**, supporting a “value trap” narrative in some cases.

---

### 🔚 Summary

* **Yield**: linked to **debt, payout, and negative momentum**.
* **Growth**: driven by **fundamentals (FCF, EPS)** and **positive technicals**.
* **Over-yielding**: may signal **underperformance or risk**, not opportunity.